In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import natsort
import glob
import numpy as np
from PIL import Image

import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1218)
if device =='cuda':
    torch.cuda.manual_seed_all(1218)

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor(),
])
train_data = torchvision.datasets.ImageFolder('your_path',transform=trans)

In [ ]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(59536, 120),
            nn.ReLU(),
            nn.Linear(120,3)
        )
        
    def forward(self, x):
        out = self.layer1(x)
#         print(out.shape) #(3,6,126,126)
        out = self.layer2(out)
#         print(out.shape) #(3,16,61,61)
        out = out.view(out.shape[0], -1)
#         print(out.shape) #(3,59536)
        out = self.layer3(out)
#         print(out.shape)
        return out

In [ ]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,256,256)).to(device)
test_out = net(test_input)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.005)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

In [ ]:
torch.save(net.state_dict(), 'your_path.pth')

In [ ]:
new_net = CNN().to(device)

In [ ]:
new_net.load_state_dict(torch.load('your_path.pth'))

In [ ]:
class CustomDataSet(Dataset):
    
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform

        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)

        return tensor_image


In [ ]:
test_data = CustomDataSet('your_path', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [ ]:
with torch.no_grad():
    file=pd.read_csv('your_path.csv')
    
    for idx, data in enumerate(test_set):
        imgs = data
        imgs = imgs.to(device)
        
        
        prediction = net(imgs)
        predict = torch.argmax(prediction, 1)
        prediction_np = predict.cpu().numpy()
        


        file['prediction'] = predict
        file.to_csv('your_path.csv', index=False)        